In [1]:
from sys import path
path.append("../..")

import warnings
warnings.filterwarnings('ignore')

In [1]:
from argparse import ArgumentParser
from numpy import percentile

from typing import Optional
from torch import device, tensor, Tensor, where
from torch.cuda import is_available as cuda_is_available
from torch_geometric.utils import remove_isolated_nodes


In [3]:
import json
import pandas as pd
from pathlib import Path
import pickle as pkl

DATA_DIR = "/home/ec2-user/lbetthauser/data/orchestration/hybrid_orchestration/security/v0"
# with open(Path(DATA_DIR, "attention_graphs.pkl"), "rb") as f:
#     graph_data = pkl.load(f)
# with open(Path(DATA_DIR, "label_mapping.json"), "rb") as f:
#     label_mapping = json.load(f)
df = pd.read_csv(open(Path(DATA_DIR, "security_intent.csv"), "rb"))

In [4]:
labels = df['reference_skill']
label_mapping = {label:i for i,label in enumerate(labels.unique())}
indexed_skills = df['reference_skill'].apply(lambda x: label_mapping[x])

In [5]:
from collections import Counter, defaultdict
# labels = Counter([graph.y for graph in graph_data if graph])
labels = Counter(indexed_skills)
smallest_number_of_examples = min([count for count in labels.values()])
TRAIN_PERCENTAGE = 0.5
MIN_NUM_EXAMPLES = 3
MAX_NUM_EXAMPLES = 275
training_counts = {label: min(max(int(labels[label]*TRAIN_PERCENTAGE),MIN_NUM_EXAMPLES),MAX_NUM_EXAMPLES) for label in labels}

train_indices = []
test_indices = []

example_count = defaultdict(list)
# for idx, graph in enumerate(graph_data):
#     if graph:
#         if len(example_count[graph.y]) < training_counts[graph.y]:
#             train_indices.append(idx)
#             example_count[graph.y].append(idx)
#         else:
#             test_indices.append(idx)

for idx, (query, skill) in enumerate(zip(df['input'], indexed_skills)):
    if query:
        if len(example_count[skill]) < training_counts[skill]:
            train_indices.append(idx)
            example_count[skill].append(idx)
        else:
            test_indices.append(idx)

# train_dataset = [graph for idx, graph in enumerate(graph_data) if idx in train_indices]
# test_dataset = [graph for idx, graph in enumerate(graph_data) if idx in test_indices]
# train_dataset = [graph for idx, graph in enumerate(graph_data) if idx in train_indices]
# test_dataset = [graph for idx, graph in enumerate(graph_data) if idx in test_indices]
        
# train_label_counter = Counter([graph.y for graph in train_dataset])
# test_label_counter = Counter([graph.y for graph in test_dataset])
train_label_counter = Counter([indexed_skills[idx] for idx in train_indices])
test_label_counter = Counter([indexed_skills[idx] for idx in test_indices])

In [6]:
train_label_counter

Counter({0: 8, 1: 5, 2: 5, 4: 5, 3: 3, 5: 3, 6: 3, 7: 3, 8: 3})

In [7]:
test_label_counter

Counter({0: 8, 1: 5, 2: 5, 4: 5, 5: 3, 6: 3, 3: 2, 7: 2, 8: 2})

In [8]:
import json
from pathlib import Path
save_path = "/home/ec2-user/lbetthauser/data/orchestration/hybrid_orchestration/security/v0"

with open(Path(save_path, "skill_map.json"), 'w') as f:
    json.dump(label_mapping, f)
# with open(Path(save_path, "train_data.pkl"), 'wb') as f:
#     pkl.dump(train_dataset, f)
# with open(Path(save_path, "test_data.pkl"), 'wb') as f:
#     pkl.dump(test_dataset, f)
df.iloc[train_indices].to_csv(Path(save_path, "train_data.csv"), index=False, header=True)
df.iloc[test_indices].to_csv(Path(save_path, "test_data.csv"), index=False, header=True)